# Chatbot legal con RAG y LangChain

Este proyecto explora cómo usar modelos de lenguaje (LLMs) junto con recuperación aumentada (RAG) para responder preguntas legales utilizando leyes mexicanas como base de conocimiento.

## Introducción

Las leyes mexicanas, como la Ley General de Protección de Datos Personales o la Ley de Transparencia, contienen información extensa y técnica. Este proyecto presenta un chatbot legal que utiliza técnicas de Recuperación Aumentada (RAG) con LangChain para ayudar a los usuarios a consultar contenido legal de manera eficiente.

A través del procesamiento de texto, creación de vectores semánticos y una interfaz conversacional, el sistema puede ofrecer respuestas informadas usando documentos legales como fuente primaria.

In [15]:
!pip install langchain langchain-community langchain-openai faiss-cpu pdfplumber > /dev/null 2>&1
!pip install pypdf > /dev/null 2>&1
!pip install gradio > /dev/null 2>&1
!wget --no-check-certificate -O LFT.pdf https://www.diputados.gob.mx/LeyesBiblio/pdf/LFT.pdf > /dev/null 2>&1

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from typing import List, Tuple
import gradio as gr
from openai import OpenAI


OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

db_path = "faiss_index"
def load_and_split_pdf(file_path):
    """Carga un archivo PDF y lo divide en fragmentos manejables."""
    loader = PyPDFLoader(file_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Tamaño de cada fragmento
        chunk_overlap=200 # Superposición entre fragmentos para mantener el contexto
    )
    chunks = text_splitter.split_documents(documents)
    return chunks

# Generar Embeddings y crear la base de datos vectorial
def create_vector_store(chunks):
    """
    Función `create_vector_store` documentada automáticamente para el portafolio.
    """
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)

    # Crear la base de datos vectorial a partir de los chunks y embeddings
    # Si ya existe, podemos cargarlo en lugar de crearlo de nuevo
    if os.path.exists(db_path):
        vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
        print("Base de datos vectorial cargada.")
    else:
        vector_store = FAISS.from_documents(chunks, embeddings)
        vector_store.save_local(db_path)
        print("Base de datos vectorial creada y guardada.")
    return vector_store


pdf_path = "LFT.pdf"

print(f"Cargando y dividiendo el documento: {pdf_path}")
text_chunks = load_and_split_pdf(pdf_path)
print(f"Número de fragmentos: {len(text_chunks)}")

print("Creando o cargando la base de datos vectorial...")
vector_db = create_vector_store(text_chunks)
print("Preparación de la base de datos vectorial completada.")

Cargando y dividiendo el documento: LFT.pdf
Número de fragmentos: 1834
Creando o cargando la base de datos vectorial...
Base de datos vectorial cargada.
Preparación de la base de datos vectorial completada.


In [11]:
def load_vector_store():
    """Carga la base de datos vectorial desde el disco"""
    embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
    vector_store = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
    return vector_store

def setup_rag_chain(vector_store):
    """Configura la cadena RAG con el LLM y la base de datos vectorial"""
    llm = ChatOpenAI(temperature=0.1, api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo")

    # Custom prompt para guiar al LLM
    template = """Eres un asistente de IA experto en la Ley Federal del Trabajo de México.
    Tu objetivo es proporcionar respuestas precisas y claras basadas exclusivamente en el contexto proporcionado.
    Si la pregunta no puede ser respondida con la información del contexto, responde que no tienes información suficiente.
    Responde en lenguaje claro y sencillo, como si explicaras a una persona sin conocimientos legales.

    Contexto: {context}
    Pregunta: {question}
    Respuesta:"""

    prompt = PromptTemplate(template=template, input_variables=["context", "question"])

    # Crear la cadena de recuperación y respuesta
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", # Combina todos los documentos recuperados en un solo prompt
        retriever=vector_store.as_retriever(),
        return_source_documents=True, # Para ver de dónde proviene la información
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain

# Simulación del chatbot

def chat_with_bot(user_query):
    """Función principal para interactuar con el chatbot"""
    # print("Cargando base de datos vectorial...")
    # vector_db = load_vector_store()
    # print("Base de datos vectorial cargada. Iniciando chatbot...")
    qa_system = setup_rag_chain(vector_db)

    print("\n¡Hola! Soy tu asistente sobre la Ley Federal del Trabajo de México. ¿En qué puedo ayudarte? (Escribe 'salir' para terminar)")


    print(f"Procesando tu pregunta: {user_query}")
    if user_query.lower() == 'salir':
        print("¡Hasta luego!")
    else:
        try:
            result = qa_system.invoke({"query": user_query})
            print("\nRespuesta del Bot:")
            print(result["result"])

            # Para debuggear las fuentes
            # print("\nDocumentos Fuente:")
            # for doc in result["source_documents"]:
            #     print(f"- Contenido: {doc.page_content[:200]}...")
            #     print(f"  Metadata: {doc.metadata}")

        except Exception as e:
            print(f"Ha ocurrido un error: {e}")

In [12]:
# TODO: Cambiar a formato {"role": ..., "content": ...} para Gradio


#variable global para almacenar el historial
conversation_history = []

def chat_with_bot(message: str, history=None):
    """Versión simplificada con memoria básica"""
    global conversation_history

    qa_system = setup_rag_chain(vector_db)

    if message.lower() == 'salir':
        conversation_history = []  # Limpiar historial al salir
        return "¡Hasta luego!"

    if message.lower() == 'limpiar':
        conversation_history = []
        return "Historial de conversación borrado."

    try:
        # Añadir contexto de las últimas 3 preguntas/respuestas
        context = "\n".join([f"Pregunta: {q}\nRespuesta: {r}" for q, r in conversation_history[-3:]])

        # Crear pregunta con contexto
        full_query = f"Contexto previo:\n{context}\n\nNueva pregunta: {message}"

        result = qa_system({"query": full_query})
        response = result["result"]

        # Guardar en el historial (máximo 5 intercambios)
        conversation_history.append((message, response))
        if len(conversation_history) > 5:
            conversation_history.pop(0)

        return response

    except Exception as e:
        return f"Error: {str(e)}"

In [13]:
def launch_chat_interface():
    """Lanza la interfaz interactiva del chatbot con disclaimer legal"""
    # Ejemplos de preguntas para mostrar en la interfaz
    examples = [
        "¿En qué casos un trabajador puede ser despedido sin responsabilidad para el patrón?",
        "¿Cuáles son las prestaciones de ley?",
        "Si me ausento por 15 días sin avisar, ¿qué pasa?"
    ]

    # Configuración de la interfaz con disclaimer legal
    disclaimer = """
    ⚠️ **AVISO LEGAL IMPORTANTE** ⚠️\n
    Este asistente virtual proporciona información general sobre la Ley Federal del Trabajo de México,
    pero **NO sustituye el asesoramiento jurídico profesional**.\n
    • No soy un abogado licenciado\n
    • Mis respuestas son generadas automáticamente basadas en el texto de la ley\n
    • Para casos específicos o asesoría legal personalizada, consulta con un profesional del derecho\n
    • El Tecnológico de Monterrey no se hace responsable por el uso que se dé a esta información\n
    \n
    ¿En qué puedo ayudarte hoy respecto a la Ley Federal del Trabajo?
    """

    demo = gr.ChatInterface(
        fn=chat_with_bot,
        title="Asistente Informativo de la LFT",
        description=disclaimer,
        examples=examples,
        theme="monochrome"
    )

    # Lanzar la interfaz
    demo.launch(share=True)

# Ejecuta esta función para probar la interfaz
launch_chat_interface()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03957376eed7fd8f9f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Interfaz Gradio

> Al ejecutar la app, Gradio generará un enlace público (ej. `https://xxxx.gradio.live`)  
> que permite probar el chatbot en el navegador.  
> Este enlace es temporal y solo funciona mientras el Colab está activo.


## Conclusión

Este proyecto demuestra cómo combinar modelos de lenguaje (LLMs) con técnicas de recuperación aumentada (RAG) para construir chatbots que pueden responder preguntas complejas basadas en documentos legales. Se implementó un flujo completo usando LangChain, FAISS y una interfaz conversacional, permitiendo consultas útiles sobre leyes mexicanas. Este enfoque puede extenderse a otros dominios como finanzas, medicina o recursos humanos.